In [ ]:
!pip install datasets

In [ ]:
import random
import time
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

# Load a smaller subset of the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1000]")

# Function to preprocess the data
def preprocess_data(data):
    # You can implement your preprocessing steps here
    preprocessed_data = data  # Placeholder for now
    return preprocessed_data

# Preprocess the dataset
train_data = preprocess_data(dataset['article'])
train_labels = dataset['highlights']

# Define different summarization pipelines
pipelines = {
    "BoW": CountVectorizer(),
    "TF-IDF": TfidfVectorizer(),
    "LDA": LatentDirichletAllocation(n_components=10),  # Assuming 10 topics
    "LSA": TruncatedSVD(n_components=10),  # Assuming 10 components
}

# Function to generate summaries using the chosen pipeline
def generate_summaries(data, pipeline):
    summaries = []
    start_time = time.time()
    for i in tqdm(range(len(data)), desc=f"Generating Summaries with {type(pipeline).__name__}"):
        # Transform the input data using the pipeline
        if isinstance(pipeline, (LatentDirichletAllocation, TruncatedSVD)):
            transformed_data = pipeline.fit_transform([data[i]])
        else:
            transformed_data = pipeline.fit_transform([data[i]]).toarray()
        # Generate summary (placeholder)
        summary = " ".join([str(x) for x in transformed_data[0]])
        summaries.append(summary)
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Summarization completed in {elapsed_time:.2f} seconds")
    return summaries

# Evaluate each pipeline on the dataset
for pipeline_name, pipeline_instance in pipelines.items():
    print(f"Evaluating pipeline: {pipeline_name}")
    # Generate summaries for the dataset
    generated_summaries = generate_summaries(train_data, pipeline_instance)
    # Calculate accuracy (placeholder)
    accuracy = accuracy_score(train_labels, generated_summaries)
    print(f"Accuracy: {accuracy:.2f}")


In [ ]:
import random
import time
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2LMHeadModel, T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm.auto import tqdm

# Load a smaller subset of the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:100]")

# Function to preprocess the data
def preprocess_data(data):
    # You can implement your preprocessing steps here
    preprocessed_data = data  # Placeholder for now
    return preprocessed_data

# Preprocess the dataset
train_data = preprocess_data(dataset['article'])
train_labels = dataset['highlights']

# Convert numerical labels to string labels
train_labels = [str(label) for label in train_labels]

# Load pre-trained BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load pre-trained GPT-2 tokenizer and model
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Load pre-trained T5 tokenizer and model
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Function to generate summaries using BERT
def generate_summaries_bert(data):
    summaries = []
    start_time = time.time()
    for i in tqdm(range(len(data)), desc="Generating Summaries with BERT"):
        inputs = bert_tokenizer(data[i], return_tensors="pt", truncation=True, padding=True)
        outputs = bert_model(**inputs)
        # Extract the pooled output, which represents the entire input sequence
        pooled_output = outputs.pooler_output
        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(pooled_output, dim=1)
        # Get the index of the maximum probability as the predicted label
        predicted_label = torch.argmax(probabilities, dim=1).item()
        summaries.append(str(predicted_label))  # Convert numerical label to string
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Summarization with BERT completed in {elapsed_time:.2f} seconds")
    return summaries

# Function to generate summaries using GPT-2
def generate_summaries_gpt(data):
    summaries = []
    start_time = time.time()
    for i in tqdm(range(len(data)), desc="Generating Summaries with GPT-2"):
        inputs = gpt_tokenizer(data[i], return_tensors="pt", max_length=512, truncation=True)
        outputs = gpt_model.generate(**inputs, max_length=512, num_return_sequences=1)
        summary = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
        summaries.append(summary)
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Summarization with GPT-2 completed in {elapsed_time:.2f} seconds")
    return summaries

# Function to generate summaries using T5
def generate_summaries_t5(data):
    summaries = []
    start_time = time.time()
    for i in tqdm(range(len(data)), desc="Generating Summaries with T5"):
        input_ids = t5_tokenizer(data[i], return_tensors="pt", max_length=512, truncation=True)['input_ids']
        summary_ids = t5_model.generate(input_ids, max_length=50, num_return_sequences=1)
        summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Summarization with T5 completed in {elapsed_time:.2f} seconds")
    return summaries

# Generate summaries for the dataset using BERT
generated_summaries_bert = generate_summaries_bert(train_data)

# Evaluate accuracy for BERT
accuracy_bert = accuracy_score(train_labels, generated_summaries_bert)
f1_bert = f1_score(train_labels, generated_summaries_bert, average='micro')
precision_bert = precision_score(train_labels, generated_summaries_bert, average='micro')
recall_bert = recall_score(train_labels, generated_summaries_bert, average='micro')
print(f"Accuracy with BERT: {accuracy_bert:.2f}")
print(f"F1 Score with BERT: {f1_bert:.2f}")
print(f"Precision with BERT: {precision_bert:.2f}")
print(f"Recall with BERT: {recall_bert:.2f}")

# Generate summaries for the dataset using GPT-2
generated_summaries_gpt = generate_summaries_gpt(train_data)

# Evaluate accuracy for GPT-2
accuracy_gpt = accuracy_score(train_labels, generated_summaries_gpt)
f1_gpt = f1_score(train_labels, generated_summaries_gpt, average='micro')
precision_gpt = precision_score(train_labels, generated_summaries_gpt, average='micro')
recall_gpt = recall_score(train_labels, generated_summaries_gpt, average='micro')
print(f"Accuracy with GPT-2: {accuracy_gpt:.2f}")
print(f"F1 Score with GPT-2: {f1_gpt:.2f}")
print(f"Precision with GPT-2: {precision_gpt:.2f}")
print(f"Recall with GPT-2: {recall_gpt:.2f}")

# Generate summaries for the dataset using T5
generated_summaries_t5 = generate_summaries_t5(train_data)

# Evaluate accuracy for T5
accuracy_t5 = accuracy_score(train_labels, generated_summaries_t5)
f1_t5 = f1_score(train_labels, generated_summaries_t5, average='micro')
precision_t5 = precision_score(train_labels, generated_summaries_t5, average='micro')
recall_t5 = recall_score(train_labels, generated_summaries_t5, average='micro')
print(f"Accuracy with T5: {accuracy_t5:.2f}")
print(f"F1 Score with T5: {f1_t5:.2f}")
print(f"Precision with T5: {precision_t5:.2f}")
print(f"Recall with T5: {recall_t5:.2f}")


In [ ]:
import random
import time
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertLMHeadModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm.auto import tqdm

# Load a smaller subset of the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:100]")

# Function to preprocess the data
def preprocess_data(data):
    # You can implement your preprocessing steps here
    preprocessed_data = data  # Placeholder for now
    return preprocessed_data

# Preprocess the dataset
train_data = preprocess_data(dataset['article'])
train_labels = dataset['highlights']

# Convert numerical labels to string labels
train_labels = [str(label) for label in train_labels]

# Load pre-trained BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertLMHeadModel.from_pretrained('bert-base-uncased', is_decoder=True)

# Function to generate summaries using BERT
def generate_summaries_bert(data):
    summaries = []
    start_time = time.time()
    for i in tqdm(range(len(data)), desc="Generating Summaries with BERT"):
        inputs = bert_tokenizer(data[i], return_tensors="pt", truncation=True, padding=True)
        max_length = min(len(inputs['input_ids'][0]), 512)  # Adjust max_length based on input data length
        outputs = bert_model.generate(**inputs, max_length=max_length, num_return_sequences=1)
        summary = bert_tokenizer.decode(outputs[0], skip_special_tokens=True)
        summaries.append(summary)
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Summarization with BERT completed in {elapsed_time:.2f} seconds")
    return summaries

# Generate summaries for the dataset using BERT
generated_summaries_bert = generate_summaries_bert(train_data)

# Print some sample predictions and ground truth labels
num_samples = 5  # Number of samples to print
print("Sample Predictions and Ground Truth Labels:")
for i in range(num_samples):
    print(f"Prediction: {generated_summaries_bert[i]}, Ground Truth: {train_labels[i]}")

# Evaluate accuracy for BERT
accuracy_bert = accuracy_score(train_labels, generated_summaries_bert)
f1_bert = f1_score(train_labels, generated_summaries_bert, average='micro')
precision_bert = precision_score(train_labels, generated_summaries_bert, average='micro')
recall_bert = recall_score(train_labels, generated_summaries_bert, average='micro')
print(f"Accuracy with BERT: {accuracy_bert:.2f}")
print(f"F1 Score with BERT: {f1_bert:.2f}")
print(f"Precision with BERT: {precision_bert:.2f}")


In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to generate a summary for the provided text
def generate_summary(text, max_summary_length):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_summary_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Function to get input from the user and generate a summary
def get_user_input():
    text = input("Enter the text you want to summarize: ")
    max_length = int(input("Enter the maximum length of the summary (in words): "))
    summary = generate_summary(text, max_length)
    print("\nSummary:")
    print(summary)

# Get input from the user and generate a summary
get_user_input()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Enter the text you want to summarize: Prime Minister Narendra Modi on Sunday took a veiled dig at Congress leader Sonia Gandhi and said people who cannot win elections fled the ground and took the Rajya Sabha route to Parliament. Addressing a public rally in Rajasthan’s Jalore, PM Modi said the country is punishing the Congress for its “sins” and the party that had once won 400 seats is unable to contest 300 seats in the Lok Sabha election 2024.  Without taking Sonia Gandhi’s name, the Prime Minister listed other senior Congress leaders who went to Rajya Sabha from Rajasthan and claimed those leaders didn’t work on the ground for the people of the state.  “Congress first sent a leader from South (KC Venugopal) to Rajya Sabha from Rajasthan. You sent Former PM Manmohan Singh to Rajya Sabha from Rajasthan. But did you see him in the state again? Another leader from the party has gone to Rajya Sabha from the state now. Those who can’t fight and win elections have come to Rajasthan this ti